In [32]:
import pandas as pd

df = pd.read_csv('test-data_encoded.csv')
df

,parsed_smell,parsed_smell_identifiers,message,message_encoded
0,\n\t\t PropertyDeclaration\n\t\t\t GenericName...,\n\t\t PropertyDeclaration\n\t\t\t GenericName...,The annotation for nullable reference types sh...,123
1,\n\t\t\t\t LocalDeclarationStatement\n\t\t\t\t...,\n\t\t\t\t LocalDeclarationStatement\n\t\t\t\t...,The annotation for nullable reference types sh...,123
2,\n\t ClassDeclaration,\n\t ClassDeclaration,Add a 'protected' constructor or the 'static' ...,3
3,\n\t ClassDeclaration,\n\t ClassDeclaration,Add a 'protected' constructor or the 'static' ...,3
4,\n\t\t MethodDeclaration\n\t\t\t IdentifierNam...,\n\t\t MethodDeclaration\n\t\t\t IdentifierNam...,"Split this method into two, one handling param...",122
...,...,...,...,...
310,\n\t\t PropertyDeclaration\n\t\t\t IdentifierN...,\n\t\t PropertyDeclaration\n\t\t\t IdentifierN...,Member 'OpenApiDocumentSample_2' does not acce...,51
311,\n\t\t\t\t ExpressionStatement\n\t\t\t\t\t Inv...,\n\t\t\t\t ExpressionStatement\n\t\t\t\t\t Inv...,Count() is used where Any() could be used inst...,20
312,\n\t\t\t\t ExpressionStatement\n\t\t\t\t\t Inv...,\n\t\t\t\t ExpressionStatement\n\t\t\t\t\t Inv...,Count() is used where Any() could be used inst...,20
313,\n\t\t\t\t ExpressionStatement\n\t\t\t\t\t Inv...,\n\t\t\t\t ExpressionStatement\n\t\t\t\t\t Inv...,"Use the ""Count"" property instead of Enumerable...",132


In [33]:
ast_list = df['parsed_smell'].tolist()
preprocessed_ast_list = [ast.strip() for ast in ast_list]

from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer('bert-base-nli-mean-tokens')

ast_embeddings = model.encode(preprocessed_ast_list, convert_to_tensor=True)
save_path = 'parsed_smell_embeddings.pt'
torch.save(ast_embeddings, save_path)

loaded_ast_embeddings = torch.load(save_path)
loaded_ast_embeddings 

tensor([[-4.2081e-02,  7.4244e-01,  6.9006e-01,  ..., -5.6429e-01,
         -2.3851e-01,  1.3245e-01],
        [-4.8465e-01,  5.6752e-01,  7.7647e-01,  ..., -6.9026e-01,
         -3.3859e-01,  5.7747e-01],
        [-2.8201e-01, -3.5818e-01,  5.6016e-01,  ..., -3.8653e-01,
         -5.2939e-01, -1.2135e-01],
        ...,
        [-8.0790e-01,  6.3382e-01,  8.0822e-01,  ..., -8.2259e-01,
         -7.3171e-01,  8.1184e-01],
        [-6.8152e-01,  4.8239e-01,  6.2383e-01,  ..., -6.0730e-01,
         -6.4566e-01,  8.6671e-01],
        [-1.0384e-03, -4.0031e-01,  1.6763e+00,  ..., -5.9387e-01,
         -4.4608e-01,  1.3031e-01]])

In [34]:
ast_list = df['parsed_smell_identifiers'].tolist()
preprocessed_ast_list = [ast.strip() for ast in ast_list]

from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer('bert-base-nli-mean-tokens')

ast_embeddings = model.encode(preprocessed_ast_list, convert_to_tensor=True)
save_path = 'parsed_smell_identifier_embeddings.pt'
torch.save(ast_embeddings, save_path)

loaded_ast_embeddings = torch.load(save_path)
loaded_ast_embeddings 

tensor([[-4.2081e-02,  7.4244e-01,  6.9006e-01,  ..., -5.6429e-01,
         -2.3851e-01,  1.3245e-01],
        [-4.6524e-01,  5.9399e-01,  8.6966e-01,  ..., -6.5632e-01,
         -3.2988e-01,  5.6402e-01],
        [-2.8201e-01, -3.5818e-01,  5.6016e-01,  ..., -3.8653e-01,
         -5.2939e-01, -1.2135e-01],
        ...,
        [-6.8476e-01,  8.8314e-01,  5.0515e-01,  ..., -7.4492e-01,
         -6.1222e-01,  7.3563e-01],
        [-5.0759e-01,  7.4027e-01,  8.5077e-01,  ..., -2.3173e-01,
         -5.6064e-01,  6.9322e-01],
        [-1.0384e-03, -4.0031e-01,  1.6763e+00,  ..., -5.9387e-01,
         -4.4608e-01,  1.3031e-01]])

In [35]:
num_embeddings = loaded_ast_embeddings.shape[0]
print(f"Number of saved embeddings: {num_embeddings}")

Number of saved embeddings: 315


In [31]:
# Example: Compute similarity between the first and second ASTs
similarity_score = util.cos_sim(ast_embeddings[6], ast_embeddings[5])
print(f"Similarity score between AST 1 and AST 2: {similarity_score}")

Similarity score between AST 1 and AST 2: tensor([[0.5824]])


In [11]:
import numpy as np

num_embeddings = len(ast_embeddings)

for ast_index in range(num_embeddings):
    similarity_scores = np.zeros(num_embeddings)

    for i in range(num_embeddings):
        if i != ast_index:
            similarity_scores[i] = util.cos_sim(ast_embeddings[ast_index], ast_embeddings[i])

    # Sort the similarity scores in descending order
    sorted_indices = np.argsort(similarity_scores)[::-1]

    # List the similarity scores for the specific AST in descending order
    print(f"Ast {ast_index}:")
    for idx in sorted_indices:
        if idx != ast_index:
            score = similarity_scores[idx]
            print(f"AST {idx}  {score:.2f}")
    print()

Ast 0:
AST 96  0.94
AST 9  0.94
AST 1  0.94
AST 210  0.93
AST 218  0.92
AST 217  0.92
AST 103  0.91
AST 102  0.91
AST 198  0.91
AST 140  0.88
AST 5  0.85
AST 135  0.84
AST 136  0.84
AST 138  0.84
AST 137  0.84
AST 145  0.84
AST 139  0.84
AST 143  0.84
AST 142  0.84
AST 146  0.84
AST 144  0.84
AST 141  0.84
AST 17  0.84
AST 38  0.82
AST 48  0.82
AST 60  0.82
AST 313  0.82
AST 133  0.82
AST 55  0.82
AST 80  0.82
AST 195  0.82
AST 22  0.82
AST 23  0.82
AST 24  0.82
AST 56  0.82
AST 50  0.82
AST 51  0.82
AST 46  0.82
AST 27  0.82
AST 25  0.82
AST 26  0.82
AST 123  0.82
AST 32  0.81
AST 72  0.81
AST 170  0.81
AST 180  0.81
AST 54  0.81
AST 52  0.81
AST 205  0.81
AST 30  0.81
AST 31  0.81
AST 21  0.81
AST 20  0.81
AST 194  0.81
AST 185  0.81
AST 183  0.81
AST 187  0.81
AST 186  0.81
AST 184  0.81
AST 181  0.81
AST 182  0.81
AST 59  0.81
AST 58  0.81
AST 47  0.81
AST 148  0.80
AST 149  0.80
AST 147  0.80
AST 215  0.80
AST 211  0.80
AST 213  0.80
AST 95  0.80
AST 74  0.80
AST 169  0.79
AST 16 

AST 230  0.74
AST 255  0.74
AST 241  0.74
AST 305  0.74
AST 298  0.74
AST 306  0.74
AST 307  0.74
AST 301  0.74
AST 300  0.74
AST 299  0.74
AST 225  0.74
AST 256  0.74
AST 264  0.74
AST 257  0.74
AST 274  0.74
AST 271  0.74
AST 270  0.74
AST 268  0.74
AST 287  0.74
AST 288  0.74
AST 267  0.74
AST 275  0.74
AST 265  0.74
AST 280  0.74
AST 258  0.74
AST 259  0.74
AST 260  0.74
AST 263  0.74
AST 262  0.74
AST 53  0.74
AST 57  0.74
AST 78  0.73
AST 77  0.73
AST 72  0.73
AST 49  0.73
AST 73  0.73
AST 119  0.73
AST 122  0.73
AST 104  0.73
AST 117  0.73
AST 111  0.73
AST 100  0.73
AST 87  0.73
AST 94  0.73
AST 98  0.73
AST 304  0.73
AST 290  0.73
AST 308  0.73
AST 284  0.73
AST 302  0.73
AST 281  0.73
AST 62  0.73
AST 234  0.73
AST 247  0.73
AST 229  0.73
AST 227  0.73
AST 231  0.73
AST 232  0.73
AST 226  0.73
AST 224  0.73
AST 261  0.73
AST 251  0.73
AST 233  0.73
AST 269  0.73
AST 35  0.73
AST 235  0.73
AST 236  0.73
AST 266  0.73
AST 228  0.73
AST 310  0.73
AST 309  0.73
AST 82  0.73
AST 8

AST 217  0.78
AST 13  0.78
AST 89  0.78
AST 88  0.78
AST 91  0.78
AST 208  0.77
AST 209  0.77
AST 7  0.77
AST 210  0.77
AST 67  0.77
AST 311  0.77
AST 312  0.77
AST 0  0.76
AST 14  0.76
AST 15  0.76
AST 132  0.76
AST 76  0.76
AST 75  0.76
AST 9  0.76
AST 313  0.76
AST 27  0.76
AST 25  0.76
AST 26  0.76
AST 1  0.75
AST 61  0.75
AST 17  0.74
AST 102  0.74
AST 103  0.74
AST 140  0.74
AST 10  0.73
AST 11  0.73
AST 33  0.72
AST 5  0.71
AST 109  0.69
AST 193  0.68
AST 8  0.68
AST 143  0.67
AST 141  0.67
AST 142  0.67
AST 139  0.67
AST 144  0.67
AST 135  0.67
AST 138  0.67
AST 137  0.67
AST 136  0.67
AST 146  0.67
AST 145  0.67
AST 113  0.65
AST 197  0.65
AST 108  0.63
AST 28  0.61
AST 29  0.61
AST 18  0.61
AST 19  0.61
AST 45  0.61
AST 6  0.37

Ast 5:
AST 139  0.91
AST 143  0.91
AST 146  0.91
AST 145  0.91
AST 135  0.91
AST 136  0.91
AST 137  0.91
AST 138  0.91
AST 144  0.91
AST 141  0.91
AST 142  0.91
AST 102  0.90
AST 103  0.90
AST 140  0.87
AST 198  0.86
AST 96  0.86
AST 0  0.85
AST 44  0

AST 219  0.22
AST 88  0.22
AST 89  0.22
AST 91  0.22

Ast 7:
AST 67  0.97
AST 312  0.97
AST 311  0.97
AST 150  0.97
AST 85  0.96
AST 15  0.96
AST 14  0.96
AST 115  0.95
AST 114  0.95
AST 205  0.95
AST 207  0.95
AST 206  0.95
AST 196  0.95
AST 13  0.94
AST 169  0.94
AST 24  0.94
AST 23  0.94
AST 22  0.94
AST 20  0.93
AST 31  0.93
AST 30  0.93
AST 21  0.93
AST 212  0.93
AST 214  0.93
AST 216  0.93
AST 219  0.93
AST 185  0.93
AST 186  0.93
AST 184  0.93
AST 183  0.93
AST 182  0.93
AST 187  0.93
AST 181  0.93
AST 170  0.93
AST 180  0.93
AST 59  0.93
AST 313  0.93
AST 34  0.92
AST 92  0.92
AST 88  0.92
AST 89  0.92
AST 91  0.92
AST 40  0.92
AST 90  0.91
AST 189  0.91
AST 79  0.91
AST 188  0.91
AST 179  0.91
AST 151  0.91
AST 152  0.91
AST 153  0.91
AST 154  0.91
AST 155  0.91
AST 158  0.91
AST 178  0.91
AST 159  0.91
AST 177  0.91
AST 160  0.91
AST 162  0.91
AST 163  0.91
AST 164  0.91
AST 165  0.91
AST 166  0.91
AST 167  0.91
AST 168  0.91
AST 156  0.91
AST 171  0.91
AST 172  0.91
AST 173 

Ast 9:
AST 210  0.97
AST 1  0.97
AST 217  0.97
AST 218  0.97
AST 0  0.94
AST 96  0.94
AST 54  0.91
AST 52  0.91
AST 198  0.91
AST 38  0.91
AST 189  0.90
AST 188  0.90
AST 90  0.90
AST 79  0.90
AST 47  0.90
AST 58  0.90
AST 55  0.90
AST 80  0.90
AST 133  0.90
AST 195  0.90
AST 211  0.90
AST 215  0.90
AST 213  0.90
AST 245  0.89
AST 244  0.89
AST 243  0.89
AST 239  0.89
AST 238  0.89
AST 46  0.89
AST 48  0.89
AST 60  0.89
AST 51  0.89
AST 50  0.89
AST 56  0.89
AST 194  0.89
AST 147  0.88
AST 148  0.88
AST 149  0.88
AST 59  0.88
AST 102  0.88
AST 103  0.88
AST 185  0.88
AST 187  0.88
AST 184  0.88
AST 183  0.88
AST 186  0.88
AST 181  0.88
AST 182  0.88
AST 313  0.88
AST 180  0.88
AST 43  0.88
AST 170  0.87
AST 236  0.87
AST 235  0.87
AST 227  0.87
AST 234  0.87
AST 233  0.87
AST 232  0.87
AST 231  0.87
AST 229  0.87
AST 228  0.87
AST 226  0.87
AST 224  0.87
AST 308  0.87
AST 247  0.87
AST 284  0.87
AST 304  0.87
AST 302  0.87
AST 269  0.87
AST 281  0.87
AST 62  0.87
AST 261  0.87
AST 35  

...

Ast 303:
AST 267  1.00
AST 278  1.00
AST 258  1.00
AST 259  1.00
AST 260  1.00
AST 262  1.00
AST 263  1.00
AST 264  1.00
AST 265  1.00
AST 268  1.00
AST 270  1.00
AST 271  1.00
AST 272  1.00
AST 273  1.00
AST 274  1.00
AST 275  1.00
AST 276  1.00
AST 257  1.00
AST 256  1.00
AST 255  1.00
AST 241  1.00
AST 221  1.00
AST 222  1.00
AST 223  1.00
AST 225  1.00
AST 230  1.00
AST 237  1.00
AST 242  1.00
AST 254  1.00
AST 246  1.00
AST 248  1.00
AST 249  1.00
AST 250  1.00
AST 252  1.00
AST 253  1.00
AST 277  1.00
AST 220  1.00
AST 279  1.00
AST 293  1.00
AST 307  1.00
AST 306  1.00
AST 305  1.00
AST 280  1.00
AST 301  1.00
AST 300  1.00
AST 298  1.00
AST 297  1.00
AST 296  1.00
AST 295  1.00
AST 294  1.00
AST 299  1.00
AST 282  1.00
AST 286  1.00
AST 289  1.00
AST 288  1.00
AST 285  1.00
AST 287  1.00
AST 283  1.00
AST 226  0.98
AST 227  0.98
AST 228  0.98
AST 281  0.98
AST 290  0.98
AST 236  0.98
AST 234  0.98
AST 233  0.98
AST 232  0.98
AST 231  0.98
AST 235  0.98
AST 304  0.98
AST 35  0.9

Ast 306:
AST 267  1.00
AST 278  1.00
AST 258  1.00
AST 259  1.00
AST 260  1.00
AST 262  1.00
AST 263  1.00
AST 264  1.00
AST 265  1.00
AST 268  1.00
AST 270  1.00
AST 271  1.00
AST 272  1.00
AST 273  1.00
AST 274  1.00
AST 275  1.00
AST 276  1.00
AST 257  1.00
AST 256  1.00
AST 255  1.00
AST 241  1.00
AST 221  1.00
AST 222  1.00
AST 223  1.00
AST 225  1.00
AST 230  1.00
AST 237  1.00
AST 242  1.00
AST 254  1.00
AST 246  1.00
AST 248  1.00
AST 249  1.00
AST 250  1.00
AST 252  1.00
AST 253  1.00
AST 277  1.00
AST 220  1.00
AST 279  1.00
AST 293  1.00
AST 307  1.00
AST 305  1.00
AST 280  1.00
AST 301  1.00
AST 300  1.00
AST 299  1.00
AST 298  1.00
AST 297  1.00
AST 296  1.00
AST 295  1.00
AST 294  1.00
AST 303  1.00
AST 289  1.00
AST 288  1.00
AST 283  1.00
AST 287  1.00
AST 282  1.00
AST 286  1.00
AST 285  1.00
AST 290  0.98
AST 281  0.98
AST 226  0.98
AST 227  0.98
AST 228  0.98
AST 235  0.98
AST 236  0.98
AST 234  0.98
AST 233  0.98
AST 232  0.98
AST 231  0.98
AST 224  0.98
AST 304  0.

AST 13  0.88
AST 7  0.88
AST 210  0.88
AST 22  0.88
AST 24  0.88
AST 23  0.88
AST 30  0.88
AST 20  0.88
AST 21  0.88
AST 31  0.88
AST 9  0.87
AST 15  0.87
AST 14  0.87
AST 92  0.87
AST 196  0.87
AST 114  0.87
AST 115  0.87
AST 40  0.87
AST 313  0.86
AST 61  0.86
AST 74  0.86
AST 16  0.86
AST 32  0.86
AST 72  0.86
AST 219  0.86
AST 216  0.86
AST 212  0.86
AST 214  0.86
AST 207  0.85
AST 206  0.85
AST 190  0.85
AST 1  0.85
AST 27  0.85
AST 26  0.85
AST 25  0.85
AST 205  0.85
AST 4  0.84
AST 17  0.84
AST 109  0.84
AST 66  0.84
AST 65  0.84
AST 64  0.84
AST 63  0.84
AST 200  0.84
AST 84  0.84
AST 82  0.84
AST 83  0.84
AST 73  0.83
AST 123  0.83
AST 95  0.83
AST 134  0.83
AST 70  0.83
AST 69  0.83
AST 12  0.83
AST 192  0.83
AST 68  0.83
AST 132  0.83
AST 76  0.83
AST 75  0.83
AST 199  0.83
AST 96  0.82
AST 111  0.82
AST 117  0.82
AST 119  0.82
AST 104  0.82
AST 122  0.82
AST 98  0.82
AST 87  0.82
AST 94  0.82
AST 100  0.82
AST 125  0.82
AST 81  0.82
AST 124  0.82
AST 120  0.82
AST 112  0.82

Ast 311:
AST 312  1.00
AST 85  0.97
AST 67  0.97
AST 7  0.97
AST 114  0.96
AST 115  0.96
AST 150  0.96
AST 206  0.96
AST 207  0.96
AST 13  0.96
AST 196  0.95
AST 205  0.95
AST 23  0.95
AST 22  0.95
AST 24  0.95
AST 92  0.95
AST 34  0.95
AST 169  0.95
AST 40  0.95
AST 214  0.94
AST 212  0.94
AST 219  0.94
AST 216  0.94
AST 21  0.94
AST 30  0.94
AST 20  0.94
AST 31  0.94
AST 313  0.94
AST 170  0.94
AST 185  0.94
AST 187  0.94
AST 181  0.94
AST 182  0.94
AST 183  0.94
AST 184  0.94
AST 186  0.94
AST 89  0.94
AST 91  0.94
AST 88  0.94
AST 180  0.94
AST 32  0.93
AST 43  0.93
AST 59  0.93
AST 190  0.92
AST 188  0.92
AST 189  0.92
AST 79  0.92
AST 90  0.92
AST 215  0.92
AST 213  0.92
AST 211  0.92
AST 171  0.92
AST 160  0.92
AST 151  0.92
AST 152  0.92
AST 153  0.92
AST 154  0.92
AST 155  0.92
AST 156  0.92
AST 158  0.92
AST 159  0.92
AST 161  0.92
AST 172  0.92
AST 162  0.92
AST 164  0.92
AST 165  0.92
AST 166  0.92
AST 167  0.92
AST 168  0.92
AST 179  0.92
AST 178  0.92
AST 177  0.92
AST 17

Ast 314:
AST 201  1.00
AST 202  1.00
AST 68  0.96
AST 200  0.95
AST 70  0.94
AST 69  0.94
AST 134  0.94
AST 4  0.93
AST 84  0.91
AST 44  0.90
AST 119  0.89
AST 122  0.89
AST 111  0.89
AST 117  0.89
AST 104  0.89
AST 87  0.89
AST 94  0.89
AST 98  0.89
AST 100  0.89
AST 83  0.88
AST 82  0.88
AST 73  0.88
AST 75  0.87
AST 76  0.87
AST 132  0.87
AST 199  0.87
AST 57  0.86
AST 53  0.86
AST 12  0.86
AST 192  0.86
AST 128  0.86
AST 107  0.85
AST 105  0.85
AST 106  0.84
AST 37  0.84
AST 130  0.84
AST 42  0.84
AST 36  0.84
AST 41  0.84
AST 131  0.84
AST 129  0.84
AST 74  0.83
AST 208  0.82
AST 209  0.82
AST 72  0.82
AST 249  0.81
AST 254  0.81
AST 253  0.81
AST 242  0.81
AST 246  0.81
AST 252  0.81
AST 250  0.81
AST 248  0.81
AST 220  0.81
AST 282  0.81
AST 279  0.81
AST 297  0.81
AST 296  0.81
AST 295  0.81
AST 294  0.81
AST 293  0.81
AST 289  0.81
AST 286  0.81
AST 285  0.81
AST 283  0.81
AST 237  0.81
AST 278  0.81
AST 277  0.81
AST 276  0.81
AST 273  0.81
AST 272  0.81
AST 221  0.81
AST 222

In [12]:
import numpy as np

num_embeddings = len(ast_embeddings)

# Choose the index of the specific AST for comparison
ast_index = 22  # Replace with the desired AST index

similarity_scores = np.zeros(num_embeddings)

for i in range(num_embeddings):
    if i != ast_index:
        similarity_scores[i] = util.cos_sim(ast_embeddings[ast_index], ast_embeddings[i])

# Sort the similarity scores in descending order
sorted_indices = np.argsort(similarity_scores)[::-1]
sorted_scores = similarity_scores[sorted_indices]

# List the similarity scores for the specific AST in descending order
print(f"Ast {ast_index}:")
for i in range(num_embeddings - 1):
    idx = sorted_indices[i]
    score = sorted_scores[i]
    print(f"AST {idx}  {score:.2f}")

Ast 22:
AST 23  1.00
AST 24  1.00
AST 30  1.00
AST 21  1.00
AST 20  1.00
AST 31  1.00
AST 32  0.97
AST 85  0.96
AST 25  0.96
AST 26  0.96
AST 27  0.96
AST 313  0.95
AST 312  0.95
AST 311  0.95
AST 114  0.95
AST 115  0.95
AST 196  0.95
AST 67  0.95
AST 150  0.94
AST 92  0.94
AST 13  0.94
AST 185  0.94
AST 187  0.94
AST 181  0.94
AST 182  0.94
AST 183  0.94
AST 184  0.94
AST 186  0.94
AST 7  0.94
AST 180  0.94
AST 169  0.94
AST 170  0.94
AST 205  0.93
AST 59  0.93
AST 40  0.93
AST 15  0.93
AST 14  0.93
AST 34  0.93
AST 43  0.92
AST 88  0.92
AST 89  0.92
AST 91  0.92
AST 46  0.91
AST 206  0.91
AST 207  0.91
AST 38  0.91
AST 214  0.91
AST 212  0.91
AST 216  0.91
AST 219  0.91
AST 80  0.91
AST 195  0.91
AST 55  0.91
AST 133  0.91
AST 149  0.91
AST 147  0.91
AST 148  0.91
AST 190  0.91
AST 211  0.90
AST 213  0.90
AST 215  0.90
AST 194  0.90
AST 156  0.90
AST 151  0.90
AST 152  0.90
AST 153  0.90
AST 154  0.90
AST 155  0.90
AST 167  0.90
AST 158  0.90
AST 168  0.90
AST 159  0.90
AST 166  0.90

In [21]:
rows_to_print = [22,5 ]
columns_to_print = ['AST']
print(df.loc[rows_to_print, columns_to_print].to_string(index=False))

                                            AST
  \n\t\t\t PredefinedType\n\t\t\t ParameterList
\n\t\t\t\t Parameter\n\t\t\t\t\t PredefinedType
